In [ ]:
# %cd /content #
!git clone https://github.com/uko3/StyleGAN-nada.git
# %cd stylegan2-pytorch
!pip install Ninja
!pip install git+https://github.com/openai/CLIP.git -q
!pip install dlib

# Создание папки для dlib моделей (если ее нет) и скачивание
%mkdir -p dlib_models
!gdown http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -O dlib_models/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk dlib_models/shape_predictor_68_face_landmarks.dat.bz2


# %cd .. #

# Создание структуры папок, если она еще не существует
%mkdir -p pretrained_models
# %mkdir -p data/inversion

# Импорты
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from argparse import Namespace
from torchvision import transforms
import dlib
import copy

# Переходим в директорию StyleGAN-nada для импорта внутренних модулей
%cd StyleGAN-nada

# Импортируем наши собственные модули
from modules.stylegan_arch.model import Generator

# from modules.utils import align_face, run_on_batch #
from modules.utils import run_on_batch #

# Зададим девайс
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используется устройство: {device}")

In [ ]:
%cd ..
!gdown https://drive.google.com/uc?id=12C0H-SE6ovcjlXruZNEO5sJhc9kDybAH -O pretrained_models/my_custom_stylegan_generator.pth

In [ ]:
# Загрузка StyleGAN-NADA Generator ()
size = 1024
latent_dim = 512
n_mlp = 8
channel_multiplier = 2
# custom_stylegan_generator_filename = 'my_custom_stylegan_generator_1.pth'

# trained_model_path = os.path.join("pretrained_models", custom_stylegan_generator_filename)
# trained_model_path = os.path.join("/kaggle/working/pretrained_models", custom_stylegan_generator_filename)
trained_model_path = "/kaggle/working/pretrained_models/my_custom_stylegan_generator_1.pth"
custom_generator = Generator(size, latent_dim, n_mlp, channel_multiplier=channel_multiplier).to(device)
custom_generator.to(device).eval() # Переводим в режим инференса

if os.path.exists(trained_model_path):
    # Если обученная модель сохранена как просто state_dict (веса модели)
    # custom_generator.load_state_dict(torch.load(trained_model_path, map_location=device))
    # Если обученная модель сохранена как чекпоинт со словарем {'g_ema': state_dict}
    checkpoint = torch.load(trained_model_path, map_location=device)
    if "g_ema" in checkpoint:
        custom_generator.load_state_dict(checkpoint["g_ema"])
        print(f"Обученная модель StyleGAN-NADA загружена из {trained_model_path} (из g_ema).")
    else:
        #
        custom_generator.load_state_dict(checkpoint) # Попытка загрузить напрямую
        print(f"Обученная модель StyleGAN-NADA загружена из {trained_model_path} (прямая загрузка).")

else:
    print(f"Ошибка: Обученная модель по пути {trained_model_path} не найдена. Убедитесь, что она была сохранена после обучения.")


# Define image transformations for pSp
transform = transforms.Compose([
    transforms.Resize((256, 256)), # pSp обычно принимает 256x256
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # Нормализация в [-1, 1]
])

In [ ]:
# второй

# Загрузка StyleGAN-NADA генераторов
size = 1024 # Выходной размер StyleGAN
latent_dim = 512
n_mlp = 8
channel_multiplier = 2

loaded_generators = {}
for name, filename in stylegan_nada_generators.items():
    # model_path = os.path.join("pretrained_models", filename) 111
    model_path = os.path.join("/kaggle/working/pretrained_models/", name)
    gen = Generator(size, latent_dim, n_mlp, channel_multiplier=channel_multiplier).to(device)
    gen.eval()

    if os.path.exists(model_path):
        checkpoint = torch.load(model_path, map_location=device)
        if "g_ema" in checkpoint:
            gen.load_state_dict(checkpoint["g_ema"])
            print(f"Генератор {name} загружен из {filename} (из g_ema).")
        else:
            gen.load_state_dict(checkpoint)
            print(f"Генератор {name} загружен из {filename} (прямая загрузка).")
        loaded_generators[name] = gen
    else:
        print(f"ПРЕДУПРЕЖДЕНИЕ: Генератор {name} по пути {model_path} не найден и не будет загружен.")

# Загрузка pSp (encoder4editing) модели
net_psp = None
try:
    ckpt_psp = torch.load(psp_model_path, map_location="cpu")
    opts_dict = ckpt_psp["opts"]
    opts_dict["checkpoint_path"] = psp_model_path
    opts = Namespace(**opts_dict)

    net_psp = pSp(opts).to(device)
    net_psp.eval()
    net_psp.load_state_dict(ckpt_psp['state_dict'], strict=False)
    print("Модель pSp (энкодер) успешно загружена!")
except Exception as e:
    print(f"ОШИБКА: Не удалось загрузить модель pSp. Проверьте путь и файл: {e}")
    net_psp = None

In [ ]:
print("\n--- Генерация случайных изображений из StyleGAN-NADA ---")
samples = 1
with torch.no_grad():
    custom_generator.to(device).eval()
    sample_z = torch.randn(samples, latent_dim, device=device)
    sample_w = custom_generator.style(sample_z) # Получаем W из Z
    sample_w_plus = sample_w.unsqueeze(1).repeat(1, custom_generator.n_latent, 1) # Преобразуем в W+ (18 слоев)

    # Генерируем изображения
    generated_images, _ = custom_generator([sample_w_plus], input_is_latent=True, randomize_noise=False)

    # Отображаем сгенерированные изображения
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    axes = axes.flatten()
    for i, img_tensor in enumerate(generated_images):
        img_np = (img_tensor.detach().cpu().clamp(-1, 1) + 1) / 2 # Нормализация в [0, 1]
        img_np = img_np.permute(1, 2, 0).numpy() #
        axes[i].imshow(img_np)
        axes[i].axis('off')
    plt.suptitle("Случайно сгенерированные изображения StyleGAN-NADA")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

print("Сгенерированы и выведены случайные изображения.")